In [1]:
using Pkg
Pkg.add(PackageSpec(name="JWAS", rev="master"))
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("CSV")
using JWAS,JWAS.Datasets,DataFrames,CSV,Statistics

    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
    Updating registry at `C:\Users\aisha\.julia\registries\General.toml`
    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`


In [2]:
phenofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/SWIM_imputation/QTLmarkers/Hormones.csv")
genofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/GWAS/dummy_genotypes.txt")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
phenotypes = dropmissing(phenotypes, :qNurlogCort)
first(phenotypes,5)

┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV C:\Users\aisha\.julia\packages\CSV\aoJqo\src\context.jl:344


Row,ID,Batch,Company,SowID,EntryAge,Cortisol,Hair1_GrindingDate,C_ExtractionDate,C_GrindingToExtractionTime,qNurlogCort,SW_144836353,SW_144841591,SW_144792892,SW_144876147,SW_144827094,SW_144841166,qNurPenBatch,SW_144783524,SW_144783527,cNurlogCort,cNurPenBatch,GrindingDate2,cNur_groundstorage
,Int64,Int64?,String1?,String15?,Int64?,Float64?,String15?,String15?,Int64?,Float64,Int64,Int64,Int64,Int64,Int64,Int64,String7?,Int64,Int64,Float64?,String7?,String15?,Int64?
1,4935,81,E,E8B260,22,16.499,4/20/2022,5/5/2022,15,2.8033,0,0,0,0,0,0,81_1,0,0,2.14908,81_D02,29/08/2022,85
2,4936,81,E,W3W953,22,13.6,4/20/2022,5/5/2022,15,2.61007,0,0,0,0,0,0,81_1,0,0,1.757,81_B04,29/08/2022,85
3,4937,81,E,E8B205,17,23.1791,4/20/2022,5/5/2022,15,3.14325,0,0,0,0,0,0,81_1,0,0,2.40153,81_D08,29/08/2022,85
4,4938,81,E,E8B411,19,16.9478,6/9/2022,5/5/2022,missing,2.83014,1,0,1,0,1,1,81_1,1,1,-0.71744,81_B04,30/08/2022,84
5,4939,81,E,3240,16,13.7074,4/22/2022,6/13/2022,52,2.61794,0,0,0,0,0,0,81_1,0,0,1.46326,81_B06,29/08/2022,135


In [3]:
MY_G3=[0.263606 0
    0 0.555353]
myPi=Dict([1.0; 1.0]=>0.000001,[1.0; 0.0]=>0.000999,[0.0; 1.0]=>0.000999,[0.0; 0.0]=>0.998001)
genotypes=get_genotypes(genofile,header=true,separator=',',MY_G3,method="BayesB",quality_control=false, Pi = myPi, estimatePi = false,estimate_scale=false);


The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095


In [8]:
# Step 3: Build Model Equations
MY_model_equations = "qNurlogCort = Batch + EntryAge  + C_GrindingToExtractionTime + SowID + qNurPenBatch + C_ExtractionDate + Hair1_GrindingDate + genotypes
             Cortisol = Batch + EntryAge  + C_GrindingToExtractionTime + SowID + qNurPenBatch + C_ExtractionDate + Hair1_GrindingDate + genotypes";
MY_R= [0.567401 0.568697
    0.568697 3.75174]
MY_model=build_model(MY_model_equations,MY_R);#

In [9]:
# Step 4: Set Factors or Covariates
set_covariate(MY_model, "EntryAge");
set_covariate(MY_model, "C_GrindingToExtractionTime");

MY_G1=[0.631587E-02 0.395474E-01
    0.395474E-01 0.591192];
set_random(MY_model,"qNurPenBatch",MY_G1); #
MY_G2=[0.631587E-02 0.395474E-01
    0.395474E-01 0.591192];
set_random(MY_model,"SowID",MY_G2); #
MY_G3=[0.631587E-02 0.395474E-01
    0.395474E-01 0.591192];
set_random(MY_model,"C_ExtractionDate",MY_G3); #
MY_G4=[0.631587E-02 0.395474E-01
    0.395474E-01 0.591192];
set_random(MY_model,"Hair1_GrindingDate",MY_G4); #



In [10]:
out=runMCMC(MY_model,phenotypes,missing_phenotypes=true,estimate_variance=true,chain_length=100,burnin=0,output_heritability=true,output_samples_frequency=10);

The folder results already exists.
The folder results1 already exists.
The folder results2 already exists.
The folder results3 already exists.
The folder results4 already exists.
The folder results5 already exists.
The folder results6 already exists.
The folder results7 already exists.
The folder results8 already exists.
The folder results9 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Missing values are found in column Batch for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column EntryAge for some observations.Effects of this variable on such observations are considere

LoadError: length of starting values for marker effects is wrong.

In [ ]:
cd("C:/Users/aisha/results1/results3")
out=GWAS(MY_model,"dummy_mapfile.txt","MCMC_samples_marker_effects_genotypes_Cortisol.txt","MCMC_samples_marker_effects_genotypes_qNurlogCort.txt",header=true,window_size="0.25 Mb",sliding_window=false,GWAS=true,genetic_correlation=true)
#sort(out, :estimate_cor, rev=true)
#CSV.write("winVarPropsBivar.txt",out)
pwd()

In [ ]:
?runMCMC